In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import urllib
from urllib.request import Request, urlopen
import json

In [2]:
# db info
dbname = 'steam_prod'
username = 'hyunjoohwang'

# make a connection to the database
conn = psycopg2.connect(f"dbname={dbname} user={username}")

In [3]:
users_games = pd.read_csv('user_games.txt', header=None, names=['game_id','play_time','user_id'], delimiter=' ')
unique_users = users_games['user_id'].unique()

In [115]:
x = 'Jan 2018'
def standardize_date(x):
    """This function converts any string date input(x) to ISO format"""
    from datetime import datetime, date
    
    try:
        if datetime.strptime(x, '%Y-%m-%d'):
            return x
    except:
        for fmt in ["%b %d, %Y", "%B %d, %Y"]:
            try:
                return datetime.strptime(x, fmt).strftime("%Y-%m-%d")
            except:
                fmt_fail = True

        for fmt in ["%b %Y", "%B %Y"]:
            try:
                d = datetime.strptime(x, fmt)
                d = d.replace(day=1)
                return d.strftime("%Y-%m-%d")
            except:
                fmt_fail = True
        
print(standardize_date(x))

2018-01-01


#### aggergate information across table and make training matrix

In [6]:
initiate empty dataframe
X = pd.DataFrame(columns=['user_id', 'dlc_id', 'parent_id', 'recommended', 'num_review', 'num_owned_games', 'dlc_play_time_hr',
       'game_play_time', 'price_diff', 'lag_month', 'dlc_price',
       'game_price', 'dlc_release_date', 'game_release_date',
       'dlc:Captionsavailable', 'dlc:Co-op', 'dlc:Commentaryavailable',
       'dlc:Cross-PlatformMultiplayer', 'dlc:DownloadableContent',
       'dlc:Fullcontrollersupport', 'dlc:Gamepad', 'dlc:HTCVive',
       'dlc:In-AppPurchases', 'dlc:IncludesSourceSDK',
       'dlc:Includesleveleditor', 'dlc:Keyboard/Mouse', 'dlc:LocalCo-op',
       'dlc:LocalMulti-Player', 'dlc:MMO', 'dlc:Multi-player',
       'dlc:OculusRift', 'dlc:OnlineCo-op', 'dlc:OnlineMulti-Player',
       'dlc:PartialControllerSupport', 'dlc:Room-Scale', 'dlc:Seated',
       'dlc:Shared/SplitScreen', 'dlc:Single-player', 'dlc:Standing',
       'dlc:Stats', 'dlc:SteamAchievements', 'dlc:SteamCloud',
       'dlc:SteamLeaderboards', 'dlc:SteamTradingCards',
       'dlc:SteamTurnNotifications', 'dlc:SteamVRCollectibles',
       'dlc:SteamWorkshop', 'dlc:TrackedMotionControllers',
       'dlc:ValveAnti-Cheatenabled', 'dlc:WindowsMixedReality',
       'Accounting', 'Action', 'Adventure', 'Animation&Modeling',
       'AudioProduction', 'Casual', 'Design&Illustration', 'EarlyAccess',
       'Education', 'FreetoPlay', 'Indie', 'MassivelyMultiplayer',
       'PhotoEditing', 'RPG', 'Racing', 'Simulation', 'SoftwareTraining',
       'Sports', 'Strategy', 'Utilities', 'VideoProduction',
       'WebPublishing', 'Captionsavailable', 'Co-op',
       'Commentaryavailable', 'Cross-PlatformMultiplayer',
       'DownloadableContent', 'Fullcontrollersupport', 'Gamedemo',
       'Gamepad', 'HTCVive', 'In-AppPurchases', 'IncludesSourceSDK',
       'Includesleveleditor', 'Keyboard/Mouse', 'LocalCo-op',
       'LocalMulti-Player', 'MMO', 'Mods', 'Mods-requireHL2',
       'Multi-player', 'OculusRift', 'OnlineCo-op', 'OnlineMulti-Player',
       'PartialControllerSupport', 'Room-Scale', 'Seated',
       'Shared/SplitScreen', 'Single-player', 'Standing', 'Stats',
       'SteamAchievements', 'SteamCloud', 'SteamLeaderboards',
       'SteamTradingCards', 'SteamTurnNotifications',
       'SteamVRCollectibles', 'SteamWorkshop', 'TrackedMotionControllers',
       'ValveAnti-Cheatenabled', 'WindowsMixedReality'])

from datetime import datetime

# grab unique users
count = 0
for user in unique_users:
    print(f"pooled {count} users")
    count += 1
    
    # SQL query to join 3 tables: review -> user -> games owned
    sql = f"""
    SELECT r.*, u.*, g.*
    FROM
         reviews r
    LEFT JOIN
         users_games u on r.final_user_id = u.user_id
    LEFT JOIN 
         games g on u.game_id = g.id 
    WHERE 
        r.final_user_id={user};
    """
    otp = pd.read_sql_query(sql, conn)
    #print(len(otp))


    drop = otp.loc[pd.isnull(otp['genres'])]
    otp.drop(drop.index, inplace=True)

    # for every game the user owns
    unique_prod = otp['product_id'].unique()
    for i in range(len(unique_prod)):
        # get product id
        prod = otp.loc[otp['product_id'] == unique_prod[i]]

        # extract info from reviews table
        num_review = prod.iloc[0]['num_reviews']
        num_owned_games = prod.iloc[0]['products']
        recommended = prod.iloc[0]['recommended']
        dlc_play_time = prod.iloc[0]['hours']

        # get game info
        game_cols = ['Accounting', 'Action', 'Adventure', 'Animation&Modeling', 'AudioProduction',
         'Casual', 'Design&Illustration', 'EarlyAccess', 'Education', 'FreetoPlay',
         'Indie', 'MassivelyMultiplayer', 'PhotoEditing', 'RPG', 'Racing', 'Simulation',
         'SoftwareTraining', 'Sports', 'Strategy', 'Utilities', 'VideoProduction',
         'WebPublishing', 'Captionsavailable', 'Co-op', 'Commentaryavailable',
         'Cross-PlatformMultiplayer', 'DownloadableContent', 'Fullcontrollersupport',
         'Gamedemo', 'Gamepad', 'HTCVive', 'In-AppPurchases', 'IncludesSourceSDK',
         'Includesleveleditor', 'Keyboard/Mouse', 'LocalCo-op', 'LocalMulti-Player',
         'MMO', 'Mods', 'Mods-requireHL2', 'Multi-player', 'OculusRift', 'OnlineCo-op',
         'OnlineMulti-Player', 'PartialControllerSupport', 'Room-Scale', 'Seated',
         'Shared/SplitScreen', 'Single-player', 'Standing', 'Stats',
         'SteamAchievements', 'SteamCloud', 'SteamLeaderboards', 'SteamTradingCards',
         'SteamTurnNotifications', 'SteamVRCollectibles', 'SteamWorkshop',
         'TrackedMotionControllers', 'ValveAnti-Cheatenabled', 'WindowsMixedReality']
        games = prod[game_cols]
        game_info_df = pd.DataFrame(data = games.values.reshape(1,len(games)), columns=games.index)

        # fetch DLC info
        dlc_id = unique_prod[i]
        sql = f"""
        select * from dlcs where dlcs.id = {dlc_id};
        """
        dlc_cols = ['Captionsavailable', 'Co-op', 'Commentaryavailable',
           'Cross-PlatformMultiplayer', 'DownloadableContent',
           'Fullcontrollersupport', 'Gamepad', 'HTCVive', 'In-AppPurchases',
           'IncludesSourceSDK', 'Includesleveleditor', 'Keyboard/Mouse',
           'LocalCo-op', 'LocalMulti-Player', 'MMO', 'Multi-player', 'OculusRift',
           'OnlineCo-op', 'OnlineMulti-Player', 'PartialControllerSupport',
           'Room-Scale', 'Seated', 'Shared/SplitScreen', 'Single-player',
           'Standing', 'Stats', 'SteamAchievements', 'SteamCloud',
           'SteamLeaderboards', 'SteamTradingCards', 'SteamTurnNotifications',
           'SteamVRCollectibles', 'SteamWorkshop', 'TrackedMotionControllers',
           'ValveAnti-Cheatenabled', 'WindowsMixedReality']
        dlc = pd.read_sql_query(sql, conn)
        if dlc.empty:
            continue
        dlc_info_df = dlc[dlc_cols]
        for col in dlc_info_df.columns:
            dlc_info_df.rename(columns={col: 'dlc:'+str(col)}, inplace=True)

        dlc_price = dlc['price']
        dlc_release_date = dlc['release_date']
        dlc_parent_id = dlc['parent_game_id'][0]


        # finally, fetch parent game info
        sql = f"""
        select * from games where games.id = {dlc_parent_id};
        """
        par_game = pd.read_sql_query(sql, conn)
        if par_game.empty:
            continue
        game_price = par_game['price']
        game_release_date = par_game['release_date']

        sql = f"""
        select * from users_games where game_id = {dlc_parent_id} and user_id = {user};
        """
        game_play_time = pd.read_sql_query(sql, conn)['play_time']

        # get price diff and time diff
        try:
            a = datetime.strptime(standardize_date(dlc_release_date[0]), "%Y-%m-%d")
            b = datetime.strptime(game_release_date[0], "%Y-%m-%d")
            lag_time = a-b
            lag = divmod(lag_time.days, 30)
            lag_month = lag[0]
            lag_day = lag[1]
        except: 
            continue
        price_diff = dlc_price - game_price

        # concatenate all information into a single row
        x = pd.concat([pd.Series(user, name='user_id'), pd.Series(dlc_id, name='dlc_id'), dlc['parent_game_id'].rename('parent_id') ,pd.Series(recommended, name='recommended'), pd.Series(num_review, name='num_review'), pd.Series(num_owned_games, name='num_owned_games'), pd.Series(dlc_play_time, name='dlc_play_time_hr'), game_play_time.rename('game_play_time'), price_diff.rename('price_diff'), pd.Series(lag_month, name='lag_month'), dlc_price.rename('dlc_price'), game_price.rename('game_price'), dlc_release_date.rename('dlc_release_date'), game_release_date.rename('game_release_date'), dlc_info_df, game_info_df], axis=1)
        # append the single row of user info to training matrix
        X = X.append(x)

pooled 0 users


/Users/hyunjoohwang/anaconda3/envs/insight/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


pooled 1 users
pooled 2 users
pooled 3 users
pooled 4 users
pooled 5 users
pooled 6 users
pooled 7 users
pooled 8 users
pooled 9 users
pooled 10 users
pooled 11 users
pooled 12 users
pooled 13 users
pooled 14 users
pooled 15 users
pooled 16 users
pooled 17 users
pooled 18 users
pooled 19 users
pooled 20 users
pooled 21 users
pooled 22 users
pooled 23 users
pooled 24 users
pooled 25 users
pooled 26 users
pooled 27 users
pooled 28 users
pooled 29 users
pooled 30 users
pooled 31 users
pooled 32 users
pooled 33 users
pooled 34 users
pooled 35 users
pooled 36 users
pooled 37 users
pooled 38 users
pooled 39 users
pooled 40 users
pooled 41 users
pooled 42 users
pooled 43 users
pooled 44 users
pooled 45 users
pooled 46 users
pooled 47 users
pooled 48 users
pooled 49 users
pooled 50 users
pooled 51 users
pooled 52 users
pooled 53 users
pooled 54 users
pooled 55 users
pooled 56 users
pooled 57 users
pooled 58 users
pooled 59 users
pooled 60 users
pooled 61 users
pooled 62 users
pooled 63 users
p

pooled 490 users
pooled 491 users
pooled 492 users
pooled 493 users
pooled 494 users
pooled 495 users
pooled 496 users
pooled 497 users
pooled 498 users
pooled 499 users
pooled 500 users
pooled 501 users
pooled 502 users
pooled 503 users
pooled 504 users
pooled 505 users
pooled 506 users
pooled 507 users
pooled 508 users
pooled 509 users
pooled 510 users
pooled 511 users
pooled 512 users
pooled 513 users
pooled 514 users
pooled 515 users
pooled 516 users
pooled 517 users
pooled 518 users
pooled 519 users
pooled 520 users
pooled 521 users
pooled 522 users
pooled 523 users
pooled 524 users
pooled 525 users
pooled 526 users
pooled 527 users
pooled 528 users
pooled 529 users
pooled 530 users
pooled 531 users
pooled 532 users
pooled 533 users
pooled 534 users
pooled 535 users
pooled 536 users
pooled 537 users
pooled 538 users
pooled 539 users
pooled 540 users
pooled 541 users
pooled 542 users
pooled 543 users
pooled 544 users
pooled 545 users
pooled 546 users
pooled 547 users
pooled 548 use

pooled 972 users
pooled 973 users
pooled 974 users
pooled 975 users
pooled 976 users
pooled 977 users
pooled 978 users
pooled 979 users
pooled 980 users
pooled 981 users
pooled 982 users
pooled 983 users
pooled 984 users
pooled 985 users
pooled 986 users
pooled 987 users
pooled 988 users
pooled 989 users
pooled 990 users
pooled 991 users
pooled 992 users
pooled 993 users
pooled 994 users
pooled 995 users
pooled 996 users
pooled 997 users
pooled 998 users
pooled 999 users
pooled 1000 users
pooled 1001 users
pooled 1002 users
pooled 1003 users
pooled 1004 users
pooled 1005 users
pooled 1006 users
pooled 1007 users
pooled 1008 users
pooled 1009 users
pooled 1010 users
pooled 1011 users
pooled 1012 users
pooled 1013 users
pooled 1014 users
pooled 1015 users
pooled 1016 users
pooled 1017 users
pooled 1018 users
pooled 1019 users
pooled 1020 users
pooled 1021 users
pooled 1022 users
pooled 1023 users
pooled 1024 users
pooled 1025 users
pooled 1026 users
pooled 1027 users
pooled 1028 users
po

...

pooled 17000 users
pooled 17001 users
pooled 17002 users
pooled 17003 users
pooled 17004 users
pooled 17005 users
pooled 17006 users
pooled 17007 users
pooled 17008 users
pooled 17009 users
pooled 17010 users
pooled 17011 users
pooled 17012 users
pooled 17013 users
pooled 17014 users
pooled 17015 users
pooled 17016 users
pooled 17017 users
pooled 17018 users
pooled 17019 users
pooled 17020 users
pooled 17021 users
pooled 17022 users
pooled 17023 users
pooled 17024 users
pooled 17025 users
pooled 17026 users
pooled 17027 users
pooled 17028 users
pooled 17029 users
pooled 17030 users
pooled 17031 users
pooled 17032 users
pooled 17033 users
pooled 17034 users
pooled 17035 users
pooled 17036 users
pooled 17037 users
pooled 17038 users
pooled 17039 users
pooled 17040 users
pooled 17041 users
pooled 17042 users
pooled 17043 users
pooled 17044 users
pooled 17045 users
pooled 17046 users
pooled 17047 users
pooled 17048 users
pooled 17049 users
pooled 17050 users
pooled 17051 users
pooled 17052

pooled 17432 users
pooled 17433 users
pooled 17434 users
pooled 17435 users
pooled 17436 users
pooled 17437 users
pooled 17438 users
pooled 17439 users
pooled 17440 users
pooled 17441 users
pooled 17442 users
pooled 17443 users
pooled 17444 users
pooled 17445 users
pooled 17446 users
pooled 17447 users
pooled 17448 users
pooled 17449 users
pooled 17450 users
pooled 17451 users
pooled 17452 users
pooled 17453 users
pooled 17454 users
pooled 17455 users
pooled 17456 users
pooled 17457 users
pooled 17458 users
pooled 17459 users
pooled 17460 users
pooled 17461 users
pooled 17462 users
pooled 17463 users
pooled 17464 users
pooled 17465 users
pooled 17466 users
pooled 17467 users
pooled 17468 users
pooled 17469 users
pooled 17470 users
pooled 17471 users
pooled 17472 users
pooled 17473 users
pooled 17474 users
pooled 17475 users
pooled 17476 users
pooled 17477 users
pooled 17478 users
pooled 17479 users
pooled 17480 users
pooled 17481 users
pooled 17482 users
pooled 17483 users
pooled 17484

pooled 17864 users
pooled 17865 users
pooled 17866 users
pooled 17867 users
pooled 17868 users
pooled 17869 users
pooled 17870 users
pooled 17871 users
pooled 17872 users
pooled 17873 users
pooled 17874 users
pooled 17875 users
pooled 17876 users
pooled 17877 users
pooled 17878 users
pooled 17879 users
pooled 17880 users
pooled 17881 users
pooled 17882 users
pooled 17883 users
pooled 17884 users
pooled 17885 users
pooled 17886 users
pooled 17887 users
pooled 17888 users
pooled 17889 users
pooled 17890 users
pooled 17891 users
pooled 17892 users
pooled 17893 users
pooled 17894 users
pooled 17895 users
pooled 17896 users
pooled 17897 users
pooled 17898 users
pooled 17899 users
pooled 17900 users
pooled 17901 users
pooled 17902 users
pooled 17903 users
pooled 17904 users
pooled 17905 users
pooled 17906 users
pooled 17907 users
pooled 17908 users
pooled 17909 users
pooled 17910 users
pooled 17911 users
pooled 17912 users
pooled 17913 users
pooled 17914 users
pooled 17915 users
pooled 17916

pooled 18296 users
pooled 18297 users
pooled 18298 users
pooled 18299 users
pooled 18300 users
pooled 18301 users
pooled 18302 users
pooled 18303 users
pooled 18304 users
pooled 18305 users
pooled 18306 users
pooled 18307 users
pooled 18308 users
pooled 18309 users
pooled 18310 users
pooled 18311 users
pooled 18312 users
pooled 18313 users
pooled 18314 users
pooled 18315 users
pooled 18316 users
pooled 18317 users
pooled 18318 users
pooled 18319 users
pooled 18320 users
pooled 18321 users
pooled 18322 users
pooled 18323 users
pooled 18324 users
pooled 18325 users
pooled 18326 users
pooled 18327 users
pooled 18328 users
pooled 18329 users
pooled 18330 users
pooled 18331 users
pooled 18332 users
pooled 18333 users
pooled 18334 users
pooled 18335 users
pooled 18336 users
pooled 18337 users
pooled 18338 users
pooled 18339 users
pooled 18340 users
pooled 18341 users
pooled 18342 users
pooled 18343 users
pooled 18344 users
pooled 18345 users
pooled 18346 users
pooled 18347 users
pooled 18348

pooled 18728 users
pooled 18729 users
pooled 18730 users
pooled 18731 users
pooled 18732 users
pooled 18733 users
pooled 18734 users
pooled 18735 users
pooled 18736 users
pooled 18737 users
pooled 18738 users
pooled 18739 users
pooled 18740 users
pooled 18741 users
pooled 18742 users
pooled 18743 users
pooled 18744 users
pooled 18745 users
pooled 18746 users
pooled 18747 users
pooled 18748 users
pooled 18749 users
pooled 18750 users
pooled 18751 users
pooled 18752 users
pooled 18753 users
pooled 18754 users
pooled 18755 users
pooled 18756 users
pooled 18757 users
pooled 18758 users
pooled 18759 users
pooled 18760 users
pooled 18761 users
pooled 18762 users
pooled 18763 users
pooled 18764 users
pooled 18765 users
pooled 18766 users
pooled 18767 users
pooled 18768 users
pooled 18769 users
pooled 18770 users
pooled 18771 users
pooled 18772 users
pooled 18773 users
pooled 18774 users
pooled 18775 users
pooled 18776 users
pooled 18777 users
pooled 18778 users
pooled 18779 users
pooled 18780

In [7]:
X.to_csv('X_full.csv')

In [105]:
# check data types
for column in X.columns:
    print(str(column) + ': ' +  str(X[column].dtype))

level_0: int64
index: int64
user_id: object
dlc_id: object
parent_id: object
recommended: object
num_review: object
num_owned_games: float64
dlc_play_time_hr: float64
game_play_time: object
price_diff: float64
lag_month: object
dlc_price: float64
game_price: float64
dlc_release_date: object
game_release_date: object
dlc:Captionsavailable: object
dlc:Co-op: object
dlc:Commentaryavailable: object
dlc:Cross-PlatformMultiplayer: object
dlc:DownloadableContent: object
dlc:Fullcontrollersupport: object
dlc:Gamepad: object
dlc:HTCVive: object
dlc:In-AppPurchases: object
dlc:IncludesSourceSDK: object
dlc:Includesleveleditor: object
dlc:Keyboard/Mouse: object
dlc:LocalCo-op: object
dlc:LocalMulti-Player: object
dlc:MMO: object
dlc:Multi-player: object
dlc:OculusRift: object
dlc:OnlineCo-op: object
dlc:OnlineMulti-Player: object
dlc:PartialControllerSupport: object
dlc:Room-Scale: object
dlc:Seated: object
dlc:Shared/SplitScreen: object
dlc:Single-player: object
dlc:Standing: object
dlc:Stats: o

In [192]:
# save column data type to preserve them for input/output
col_dtype = {'user_id': object,
'dlc_id': float,
'parent_id': float,
'recommended': float,
'num_review': float,
'num_owned_games': float,
'dlc_play_time_hr': float,
'game_play_time': float,
'price_diff': float,
'lag_month': float,
'dlc_price': float,
'game_price': float,
'dlc_release_date': object,
'game_release_date': object,
'dlc:Captionsavailable': float,
'dlc:Co-op': float,
'dlc:Commentaryavailable': float,
'dlc:Cross-PlatformMultiplayer': float,
'dlc:DownloadableContent': float,
'dlc:Fullcontrollersupport': float,
'dlc:Gamepad': float,
'dlc:HTCVive': float,
'dlc:In-AppPurchases': float,
'dlc:IncludesSourceSDK': float,
'dlc:Includesleveleditor': float,
'dlc:Keyboard/Mouse': float,
'dlc:LocalCo-op': float,
'dlc:LocalMulti-Player': float,
'dlc:MMO': float,
'dlc:Multi-player': float,
'dlc:OculusRift': float,
'dlc:OnlineCo-op': float,
'dlc:OnlineMulti-Player': float,
'dlc:PartialControllerSupport': float,
'dlc:Room-Scale': float,
'dlc:Seated': float,
'dlc:Shared/SplitScreen': float,
'dlc:Single-player': float,
'dlc:Standing': float,
'dlc:Stats': float,
'dlc:SteamAchievements': float,
'dlc:SteamCloud': float,
'dlc:SteamLeaderboards': float,
'dlc:SteamTradingCards': float,
'dlc:SteamTurnNotifications': float,
'dlc:SteamVRCollectibles': float,
'dlc:SteamWorkshop': float,
'dlc:TrackedMotionControllers': float,
'dlc:ValveAnti-Cheatenabled': float,
'dlc:WindowsMixedReality': float,
'Accounting': float,
'Action': float,
'Adventure': float,
'Animation&Modeling': float,
'AudioProduction': float,
'Casual': float,
'Design&Illustration': float,
'EarlyAccess': float,
'Education': float,
'FreetoPlay': float,
'Indie': float,
'MassivelyMultiplayer': float,
'PhotoEditing': float,
'RPG': float,
'Racing': float,
'Simulation': float,
'SoftwareTraining': float,
'Sports': float,
'Strategy': float,
'Utilities': float,
'VideoProduction': float,
'WebPublishing': float,
'Captionsavailable': float,
'Co-op': float,
'Commentaryavailable': float,
'Cross-PlatformMultiplayer': float,
'DownloadableContent': float,
'Fullcontrollersupport': float,
'Gamedemo': float,
'Gamepad': float,
'HTCVive': float,
'In-AppPurchases': float,
'IncludesSourceSDK': float,
'Includesleveleditor': float,
'Keyboard/Mouse': float,
'LocalCo-op': float,
'LocalMulti-Player': float,
'MMO': float,
'Mods': float,
'Mods-requireHL2': float,
'Multi-player': float,
'OculusRift': float,
'OnlineCo-op': float,
'OnlineMulti-Player': float,
'PartialControllerSupport': float,
'Room-Scale': float,
'Seated': float,
'Shared/SplitScreen': float,
'Single-player': float,
'Standing': float,
'Stats': float,
'SteamAchievements': float,
'SteamCloud': float,
'SteamLeaderboards': float,
'SteamTradingCards': float,
'SteamTurnNotifications': float,
'SteamVRCollectibles': float,
'SteamWorkshop': float,
'TrackedMotionControllers': float,
'ValveAnti-Cheatenabled': float,
'WindowsMixedReality': float}

In [182]:
# check to see data type is preserved
newX = pd.read_csv('X_full_raw.csv', dtype=col_dtype, index_col=0)

In [193]:
newX.head()
newX = newX.reset_index(drop=True)

In [194]:
len(newX)
newX.head(5)

,user_id,dlc_id,parent_id,recommended,num_review,num_owned_games,dlc_play_time_hr,game_play_time,price_diff,lag_month,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
0,76561197999899413,760610.0,744800.0,1.0,6.0,888.0,0.0,6.0,2.99,0.0,...,1287.0,728.0,375.0,1089.0,7.0,0.0,98.0,7.0,28.0,0.0
1,76561198151921721,760610.0,744800.0,1.0,37.0,213.0,0.0,5.0,2.99,0.0,...,190.0,111.0,53.0,156.0,0.0,1.0,26.0,3.0,4.0,1.0
2,76561198141277447,779280.0,776490.0,1.0,8.0,164.0,0.0,617.0,0.00,0.0,...,124.0,68.0,17.0,109.0,0.0,1.0,12.0,2.0,12.0,0.0
3,76561198067653526,779280.0,776490.0,1.0,94.0,304.0,0.0,1371.0,0.00,0.0,...,273.0,165.0,105.0,227.0,3.0,0.0,42.0,1.0,9.0,0.0
4,76561198067653526,316910.0,259680.0,1.0,94.0,304.0,0.0,10044.0,-3.00,22.0,...,273.0,165.0,105.0,227.0,3.0,0.0,42.0,1.0,9.0,0.0


In [195]:
X=newX

### inspect data
- one obvious column (based on intuition) I could check is the price diff
    - not many DLCs are more expensive than their parent game

In [196]:
X.loc[X['price_diff'] > 10,['dlc_id','parent_id','dlc_price','game_price','price_diff']].head(2)

,dlc_id,parent_id,dlc_price,game_price,price_diff
11,8650.0,8600.0,39.99,3.99,36.0
12,8650.0,8600.0,39.99,3.99,36.0


#### manually inspect game prices and replace incorrect values to correct ones

In [197]:
# inspecting data, some obvious price mismatches - dlc_id: 8650. Others look correct
X.loc[X['dlc_id'] == 8650, 'dlc_price'] = 0.0
X.loc[X['dlc_id'] == 8650, 'price_diff'] = X.loc[X['dlc_id'] == 8650, 'dlc_price'] - X.loc[X['dlc_id'] == 8650, 'game_price']

In [198]:
dlcid = 639600
X.loc[X['dlc_id'] == dlcid, 'dlc_price'] = 0.0
X.loc[X['dlc_id'] == dlcid, 'price_diff'] = X.loc[X['dlc_id'] == dlcid, 'dlc_price'] - X.loc[X['dlc_id'] == dlcid, 'game_price']

In [199]:
gameid = 389280
X.loc[X['dlc_id'] == gameid, 'game_price'] = 0.0
X.loc[X['dlc_id'] == gameid, 'price_diff'] = X.loc[X['dlc_id'] == gameid, 'dlc_price'] - X.loc[X['dlc_id'] == gameid, 'game_price']

In [200]:
X.loc[X['dlc_id'] == 701310]

,user_id,dlc_id,parent_id,recommended,num_review,num_owned_games,dlc_play_time_hr,game_play_time,price_diff,lag_month,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
7891,76561198051090674,701310.0,333930.0,1.0,57.0,122.0,0.0,19067.0,19.99,27.0,...,81.0,45.0,17.0,80.0,0.0,0.0,31.0,1.0,22.0,0.0
19422,76561198314271203,701310.0,333930.0,0.0,11.0,18.0,0.0,2966.0,19.99,27.0,...,19.0,4.0,2.0,17.0,0.0,0.0,6.0,1.0,7.0,0.0
19423,76561198024772951,701310.0,333930.0,0.0,1.0,16.0,0.0,46482.0,19.99,27.0,...,7.0,3.0,0.0,7.0,0.0,0.0,1.0,0.0,2.0,0.0
19424,76561197999716524,701310.0,333930.0,0.0,1.0,92.0,0.0,3111.0,19.99,27.0,...,46.0,29.0,16.0,41.0,0.0,0.0,16.0,0.0,7.0,0.0


In [201]:
# #X = X.reset_index()
# # dlc_drop = [701312,701310]
X.drop(X.loc[X['dlc_id'] == 701310].index, inplace=True)
X.drop(X.loc[X['dlc_id'] == 701312].index, inplace=True)

In [202]:
len(X)

21896

In [223]:
dlc_ok = [8650,411894,411930,649271,649270,651130,640070,537324,557350,551880,448181,547230,468190,460030,462920
         ,403350,277777,358144,266590,316960,244230,113405,712340216468.0,524330.0]
parent_ok = [24010,311730,253370,223750,236390,113400,312450,224600,253710,304050,238260,314160,244050
            ,286940,273110,291480,380600,552110,444640,215280,389280,206210.0,504370.0,301520.0,327690.0,648430.0
            ,291650.0,378480.0,607200.0,627690.0,621880.0,550900.0,310380.0,342200.0,426690.0,329110.0
            ,389040.0,339280.0,345010.0,342200.0,389040.0,292730.0,386180.0,216150.0]

#  & (~X['parent_id'].isin(parent_ok))
X.loc[(X['price_diff'] > 10) & (~X['dlc_id'].isin(dlc_ok)) & (~X['parent_id'].isin(parent_ok)), ['dlc_id','parent_id','dlc_price','game_price','price_diff']]

,dlc_id,parent_id,dlc_price,game_price,price_diff


#### inspect other obvious things like missing y values

In [225]:
X.drop(X.loc[pd.isnull(X['recommended']) == True].index, inplace=True)

In [227]:
X.loc[pd.isnull(X['game_play_time']) == True].head(2)

,user_id,dlc_id,parent_id,recommended,num_review,num_owned_games,dlc_play_time_hr,game_play_time,price_diff,lag_month,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
57,76561198133031298,8650.0,8600.0,0.0,1.0,9.0,0.4,NaN,-3.99,8.0,...,16.0,10.0,3.0,12.0,0.0,0.0,5.0,0.0,3.0,0.0
59,76561198025453989,8650.0,8600.0,1.0,11.0,99.0,0.0,NaN,-3.99,8.0,...,92.0,48.0,13.0,69.0,0.0,0.0,27.0,1.0,22.0,0.0


In [228]:
# drop NaN for game play time because this means user doesn't own the parent game of the dlc
X.drop(X.loc[pd.isnull(X['game_play_time']) == True].index, inplace=True)

In [229]:
print(X.shape)
X.head()

(21413, 111)


,user_id,dlc_id,parent_id,recommended,num_review,num_owned_games,dlc_play_time_hr,game_play_time,price_diff,lag_month,...,SteamAchievements,SteamCloud,SteamLeaderboards,SteamTradingCards,SteamTurnNotifications,SteamVRCollectibles,SteamWorkshop,TrackedMotionControllers,ValveAnti-Cheatenabled,WindowsMixedReality
0,76561197999899413,760610.0,744800.0,1.0,6.0,888.0,0.0,6.0,2.99,0.0,...,1287.0,728.0,375.0,1089.0,7.0,0.0,98.0,7.0,28.0,0.0
1,76561198151921721,760610.0,744800.0,1.0,37.0,213.0,0.0,5.0,2.99,0.0,...,190.0,111.0,53.0,156.0,0.0,1.0,26.0,3.0,4.0,1.0
2,76561198141277447,779280.0,776490.0,1.0,8.0,164.0,0.0,617.0,0.00,0.0,...,124.0,68.0,17.0,109.0,0.0,1.0,12.0,2.0,12.0,0.0
3,76561198067653526,779280.0,776490.0,1.0,94.0,304.0,0.0,1371.0,0.00,0.0,...,273.0,165.0,105.0,227.0,3.0,0.0,42.0,1.0,9.0,0.0
4,76561198067653526,316910.0,259680.0,1.0,94.0,304.0,0.0,10044.0,-3.00,22.0,...,273.0,165.0,105.0,227.0,3.0,0.0,42.0,1.0,9.0,0.0


In [230]:
X.to_csv('X_full_cleaned.csv')